In [7]:
import os 
import pandas as pd 
import re


# Load data

In [2]:
train_path = "../data/piqa/piqa_train.json"
val_path = "../data/piqa/piqa_validation.json"
test_path = "../data/piqa/piqa_test.json"

train_df = pd.read_json(train_path)
val_df = pd.read_json(val_path)
test_df = pd.read_json(test_path)




In [4]:
train_df.head()

,goal,sol1,sol2,label
0,"When boiling butter, when it's ready, you can",Pour it onto a plate,Pour it into a jar,1
1,"To permanently attach metal legs to a chair, y...",Weld the metal together to get it to stay firm...,Nail the metal together to get it to stay firm...,0
2,how do you indent something?,leave a space before starting the writing,press the spacebar,0
3,how do you shake something?,move it up and down and side to side quickly.,stir it very quickly.,0
4,Clean tires,"Pour water, cape off caked on dirt. Use speed...","Pour water, scrape off caked on dirt. Use a st...",1


# Prompt

In [6]:
with open("prompt.txt", "r") as file:
    prompt = file.read()
    
print(prompt)



You are an intelligent assistant that helps evaluate practical solutions to everyday tasks and problems. Your role is to analyze solutions in a given context and determine which one is more effective and practical.

Given a context and two possible solutions, choose the better solution that makes more sense for that context.

Here are some important rules for the task:
- Think carefully and logically about which solution is more appropriate for the given context
- Consider practicality, safety, and effectiveness when evaluating the solutions
- You must explain your reasoning for why your chosen solution is better
- You must answer with <answer>0</answer> if Solution 1 is better, or <answer>1</answer> if Solution 2 is better
- Put your final answer in <answer></answer> tags after your explanation

Here are some examples:
<example>

<question>
Based on the context, which solution is better?
Context: When boiling butter, when it's ready, you can
Solution 1: Pour it onto a plate
Solution 2

# Gemini

In [ ]:
# ... existing imports ...
from transformers import AutoTokenizer, AutoModelForCausalGeneration
import torch

# Load model and tokenizer
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalGeneration.from_pretrained(model_name)

# Move model to GPU if available
if torch.cuda.is_available():
    device = [0, 1]  # Use GPUs 0 and 1
    model = torch.nn.DataParallel(model, device_ids=device)
    model = model.to(f'cuda:{device[0]}')
else:
    device = "cpu"
    model = model.to(device)

def post_process(response):
    pattern = r'<answer>(.*?)</answer>'
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    return None


def get_llama_response(row):
    # Format the prompt with the current example
    formatted_prompt = prompt.format(
        goal=row['goal'],
        sol1=row['sol1'], 
        sol2=row['sol2']
    )
    
    # Tokenize and generate
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device if isinstance(device, str) else f'cuda:{device[0]}')
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode and return response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


# Evaluate

In [ ]:
res = []
for idx, row in val_df.iterrows():
    response = get_llama_response(row)
    final_answer = post_process(response)
    res.append(final_answer) 
    
val_df['llama_answer'] = res




In [ ]:
val_df.to_csv("llama_eval.csv", index=False)


In [ ]:
accuracy = (val_df['llama_answer'] == val_df['label']).mean()
print(f"Accuracy: {accuracy:.2f}")

